# Installing required libraries

In [1]:
!pip install pypdf cryptography

# Import libraries and set up current date

In [2]:
from pypdf import PdfReader, PdfWriter
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from datetime import datetime

# Create digital stamp text

In [3]:
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
stamp_text = f"Digitally signed by R. Haase, {current_time}"

# Generate key pair for signing

In [4]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)

# Add stamp and signature to PDF

In [5]:
reader = PdfReader("test2.pdf")
writer = PdfWriter()

# Get first page
page = reader.pages[0]
writer.add_page(page)

# Add stamp annotation
writer.add_annotation(
    page_number=0,
    annotation={
        "type": "/FreeText",
        "rect": [50, 50, 400, 70],
        "contents": stamp_text,
        "color": [1, 0, 0]  # RGB for red
    }
)

# Add signature field
writer.add_form_field(
    name="Signature",
    typ="/Sig",
    rect=(50, 20, 400, 40),
    value="Cryptographically signed by R. Haase"
)

AttributeError: 'PdfWriter' object has no attribute 'add_form_field'

# Sign and save the PDF

In [ ]:
# Create hash and sign
hash_pdf = hashes.Hash(hashes.SHA256())
pdf_bytes = writer.bytes()
hash_pdf.update(pdf_bytes)
signature = private_key.sign(
    hash_pdf.finalize(),
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

# Save the signed PDF
with open("signed_test2.pdf", "wb") as output_file:
    writer.write(output_file)